In [18]:
import eikon as ek
import pandas as pd
from dataquery import *

ek.set_app_key('YOUR_APP_KEY')

We get all global vars and methods from dataquery into this namespace, so we do not need to write code like dataquery.SCREEN, etc.

The below cell shows, how the user can create the screener query in 1 line of code via the "express" class:

In [19]:
screen = SCREEN.express.universe(Equity(active=True, public=True, primary=True)) \
                .conditions(IN('TR.CoRTradingCountryCode', 'US'), TOP('TR.CompanyMarketCap', 10, nnumber)) \
                .currency('USD').query

screen

'SCREEN(U(IN(Equity(primary,public,active))),IN(TR.CoRTradingCountryCode,US),TOP(TR.CompanyMarketCap,10,nnumber),CURN=USD)'

The user can also get & set the parameters like universe, conditions, currency by setting them as properties directly in the SCREEN class:

In [20]:
SCREEN.universe

'Equity(primary,public,active)'

In [21]:
SCREEN.query

'SCREEN(U(IN(Equity(primary,public,active))),IN(TR.CoRTradingCountryCode,US),TOP(TR.CompanyMarketCap,10,nnumber),CURN=USD)'

<br>
<br>
We can now use the SCREEN statement in the get_data method with Eikon Data API:

In [22]:
df, err = ek.get_data(screen, ['TR.CompanyMarketCap'])
df

,Instrument,Company Market Cap
0,JNJ.N,3.721365e+11
1,WMT.N,3.343971e+11
2,AMZN.OQ,1.283191e+12
3,AAPL.OQ,1.486634e+12
4,MSFT.OQ,1.432815e+12
5,BRKa.N,4.409172e+11
6,FB.OQ,6.624165e+11
7,V.N,4.082607e+11
8,JPM.N,3.085111e+11
9,GOOGL.OQ,9.695267e+11


Here is another query example:

In [23]:
screen2 = SCREEN.express \
            .universe('0#.SPX') \
            .conditions(IN('TR.TRBCEconomicSector', 'Technology')) \
            .currency('USD').query

Let's add the ADC function "Rel" to pull the relative performance of the filtered S&P 500 stocks vs Apple:

In [24]:
df2, err = ek.get_data(screen2, ['TR.EpsSmartEst', REL('AAPL.O','TR.EpsSmartEst')])
df2

,Instrument,Earnings Per Share - SmartEstimate®,"REL(AAPL.O,TR.EPSSMARTEST)"
0,QRVO.OQ,5.51812,0.446105
1,AMD.OQ,1.03433,0.083619
2,WDC.OQ,3.00762,0.243147
3,ZBRA.OQ,10.60356,0.857231
4,AMAT.OQ,3.81228,0.308199
...,...,...,...
67,ACN.N,7.59153,0.613727
68,CTXS.OQ,5.62087,0.454412
69,FLIR.OQ,2.07179,0.167491
70,DXC.N,2.99921,0.242467


We can also specify the screener to look into M&A deals:

In [25]:
screen3 = SCREEN.express.universe(DEALS).conditions(IN('TR.MnAStatus', 'C'), RELATIVEDATE('TR.MnARelatedAnnDate', 'LM')).currency('USD').query

In [26]:
df3, err = ek.get_data(screen3, ['TR.MNASDCDealNumber', 'TR.MnARelatedAcquiror', 'TR.MnADealValue'])
df3

,Instrument,SDC Deal No,Related M&A Deal Acquiror,Deal Value
0,34392064565,2.324115e+09,Genesys Telecommunications Laboratories Inc SPV,1.500000e+09
1,154085530572,3.476810e+09,Warner Media LLC,NaN
2,154085679613,3.571192e+09,Undisclosed Acquiror,6.000000e+07
3,154085679614,3.571193e+09,Undisclosed Acquiror,NaN
4,154085680262,3.571493e+09,Veregy,NaN
...,...,...,...,...
100,154085761636,NaN,Public Investment Fund,NaN
101,154085761636,NaN,Public Investment Fund,NaN
102,154085763472,3.587652e+09,Solina Group SAS,NaN
103,154085765568,3.588569e+09,SCR Sibelco NV,NaN


The rest of this file contains more examples:

In [27]:
screen4 = SCREEN.express.universe('0#.SPX').conditions(TOP('TR.CompanyMarketCap', 10, nnumber)).currency('USD').query

In [28]:
df4, err = ek.get_data(screen4, ['TR.PriceClose', ROUND(FORMULA('TR.PriceClose', '*2'),0)])
df4

,Instrument,Price Close,"ROUND(TR.PriceClose*2,0)"
0,AAPL.OQ,342.99,686
1,MSFT.OQ,188.94,378
2,AMZN.OQ,2572.68,5145
3,GOOGL.OQ,1420.74,2841
4,GOOG.OQ,1419.85,2840
5,FB.OQ,232.50,465
6,BRKb.N,181.55,363
7,V.N,191.76,384
8,JNJ.N,141.25,282
9,WMT.N,118.08,236


In [29]:
df5, err = ek.get_data('0#.SPX', ['TR.EPSMean', ZSCORE('TR.Close', '1M')])
df5

,Instrument,Earnings Per Share - Mean,(TR.CLOSE-AVG(TR.CLOSE(EDATE=-1M)))/STD(TR.Close(edate=-1M))
0,CHRW.OQ,2.88023,0.218734
1,AJG.N,3.93546,0.624372
2,CNP.N,1.30764,0.871579
3,AMCR.N,0.63429,0.918763
4,WM.N,3.63264,0.338907
...,...,...,...
500,DXC.N,2.89629,-0.124185
501,SNPS.OQ,5.26873,1.118065
502,J.N,4.95464,0.294399
503,SIVB.OQ,12.69875,0.811618
